


Objectifs:
Savoir organiser des données en pages pour permettre de modifier un tuple en ne modifiant qu'une seule page.

Comprendre les méthodes d'accès suivantes :

*   Lecture séquentielle d'une fichier : "table access full"
*   Lecture d'un tuple dont on connait le rowid : "table access by index rowid"
*   Opération de sélection par lecture séquentielle et filtrage

Comprendre les méthodes d'indexation :

*   Créer un index
*   Opération de sélection par index et lecture par rowid

Mise à jour de données
*   Sélectionner un tuple et modifier un de ses attributs
*   Modifier l'index en conséquence lorsque l'attibut modifié est indexé

Persistence
*   Stocker un index (dans plusieurs pages) pour le reconstruire plus rapidement
*   Adapter en conséquence les opérations de modification de l'index


# TP 1 et 2 : Accès aux données avec index
# sujet

date de modification : 18/01/2024

BINOME

NOM 1: ZHOU

Prénom 1: runlin

Numéro : 28717281

NOM 2: ZHANG 

Prénom 2: zhile

Numéro : 21201131

TP à rendre : **REDIGER des explications détaillées et argumentées** pour les solutions que vous proposez

In [58]:
import os
import shutil as sh
import numpy as np
import pandas as pd
import random

# from sortedcontainers import SortedDict
import sortedcontainers

from string import ascii_lowercase
import time

# le nom de la table
TABLE = "T"
print("le nom de la table est", TABLE)


# le nom du fichier qui contient les données de la table
def nom_fichier(table):
    return table + ".csv"

le nom de la table est T


# Générer les données du TP

Création du fichier contenant un exemple de données.
Ce sont des données au format csv. On suppose que chaque ligne correspond à un tuple d'une table **T** ayant *n* attributs :

$$ T (a_0, a_1, a_2, ..., a_{n-1})$$

Le premier attribut $a_0$ est unique.

Les attributs $a_1$ à $a_{n-2}$ ne sont pas uniques : il y a en moyenne $2^k$ tuples par valeurs de $a_k$ soit 2 tuples par valeur de $a_1$ et 4 tuples par valeurs de $a_2$.

Les attributs sont des nombres entiers, multiples de 10, sauf le dernier qui est une chaine de caractères (on choisit des mutliples de 10 pour représenter le cas plus général de domaines contenant des valeurs non consécutives).



In [59]:
# dure environ 20s pour 2M lignes
# dure environ 40s pour 5M lignes


def genere_fichier(nb_lignes, nb_attributs, longueur_dernier_attribut, table):
  # attribut_chaine_caracteres = "".join(choice(ascii_lowercase) for i in range(longueur_dernier_attribut))
  attribut_chaine_caracteres = ''.join('-' for i in range(longueur_dernier_attribut))
  # print("le dernier attribut de chaque tuple est la chaine de caracètes :", attribut_chaine_caracteres)

  # reproductibilité des données générées
  rng = np.random.default_rng(seed=1)

  data={}

  # le premier attribut est unique.
  nb_valeurs_distinctes = nb_lignes
  data['a0'] = 10 * rng.permutation(np.arange(nb_valeurs_distinctes))

  # les attributs suivants ont des domaines plus petits :
  for i in range(1, nb_attributs):
    # on divise le domaine par 2 à chaque itération
    nb_valeurs_distinctes = max(2, int(nb_valeurs_distinctes / 2))
    data[f'a{i}'] = 10 * rng.integers(0, nb_valeurs_distinctes, nb_lignes)

  # on concatène "verticalement" les attributs dans un dataframe pour former des tuples sur lesquels on peut itérer.
  df = pd.DataFrame(data)
  # rmq: le dernier attribut est une chaine de caractères
  b = [str(e)[1:-1] + f",{attribut_chaine_caracteres}\n" for e in df.itertuples(index=False, name=None)]

  # on stocke les données dans un fichier
  fichier = nom_fichier(table)
  print("écriture des données dans le fichier", fichier)
  with open(fichier, "w") as f:
    # écriture groupée de tous les tuples
    f.write(''.join(b))

nb_lignes = 2 * 1000 * 1000
# nb_lignes = 5 * 1000 * 1000
nb_attributs = 7
longueur_dernier_attribut = 100

t1 = time.time()
genere_fichier(nb_lignes, nb_attributs, longueur_dernier_attribut, TABLE)
print(f"durée pour générer {nb_lignes} lignes :", round(time.time() - t1, 1), "s")

écriture des données dans le fichier T.csv
durée pour générer 2000000 lignes : 5.8 s


On affiche le début et la fin du fichier et son nombre de lignes ( = card(T))

In [60]:
%%bash -s "{TABLE}.csv"
echo "debut de $1 :"
head -n 2 $1
echo
echo "fin de $1 : "
tail -n 2 $1
echo
echo "nombre de lignes:"
wc -l $1


debut de T.csv :
19512870, 5751570, 1235270, 1344040, 1139220, 359300, 261790,----------------------------------------------------------------------------------------------------
3462320, 3047440, 4558650, 2392580, 332870, 445910, 35390,----------------------------------------------------------------------------------------------------

fin de T.csv : 
4418430, 9611540, 268500, 660850, 881140, 311230, 273070,----------------------------------------------------------------------------------------------------
1896950, 9749280, 4493840, 1757100, 1083240, 323240, 177530,----------------------------------------------------------------------------------------------------

nombre de lignes:
 2000000 T.csv


# Lecture séquentielle

On définit un *iterateur* pour lire séquentiellement chaque ligne de la table stockée entièrement dans un seul fichier.
Le mot python *yield* permet de définir un itérateur qui est retourné par la fonction.

Cet itérateur est invoqué pour lire la table et appliquer un filtre.



In [61]:
def lecture_sequentielle(table):
  fichier = nom_fichier(table)
  with open(fichier, "r") as f:
    for i, line in enumerate(f):
      yield i, line

def filtrer_table(table, valeur_recherchee):
  for i, line in lecture_sequentielle(table):
      a = int(line.split(',')[0])
      if a == valeur_recherchee :
        print(f"ligne {i} :", line.strip())


nb_valeurs_distinctes = nb_lignes
s = np.random.randint(nb_valeurs_distinctes)
print("valeur recherchée :", s)

t1 = time.time()
filtrer_table(TABLE, s)
print("durée :", round(time.time() - t1, 3), "s")

valeur recherchée : 1798348
durée : 2.135 s


# Découper une table en pages

On organise les données en pages.
Pour faciliter le TP, chaque page est représentée par un "petit" fichier mais en réalité une page est un bloc d'un fichier.

Dans la suite du TP, on accédera toujours aux pages.
Le fichier créé initialement, contenant tous les tuples, ne sera plus utilisé.

In [62]:
def page_dir_name(table):
  return table + "_pages"

def decoupe_table_en_pages(table, nb_tuple_par_page):
  page_dir = page_dir_name(table)

  # vider le dossier qui contiendra les pages
  if(os.path.exists(page_dir)):
    sh.rmtree(page_dir)
  os.makedirs(page_dir, exist_ok=True)

  # lire le fichier contenant tous les tuples
  p=0
  lines = []
  for i, line in lecture_sequentielle(table):
    lines.append(line)
    if (i+1) % nb_tuple_par_page == 0:

      # créer une page
      p += 1
      with open(page_dir + f"/page{p}", "w") as fp:
        fp.write(''.join(lines))
      lines = []

  # créer une dernière page, si nécessaire
  if len(lines) > 0:
    p +=1
    with open(page_dir + f"/page{p}", "w") as fp:
        fp.write(''.join(lines))

  print("nb pages créées :", p)


print("les pages sont stockées dans le dossier", page_dir_name(TABLE) )

decoupe_table_en_pages(TABLE, nb_tuple_par_page=1000)

les pages sont stockées dans le dossier T_pages
nb pages créées : 2000


Afficher (pour quelques pages) le nombre de tuples contenus dans une page

une solution en bash :

In [63]:
%%bash -s "$TABLE"
wc -l $1_pages/* | head -n 3

    1000 T_pages/page1
    1000 T_pages/page10
    1000 T_pages/page100


une autre solution en python :

In [64]:
page_dir = page_dir_name(TABLE)
l = os.listdir(page_dir)
random.seed(1)
for i in range(3):
  une_page = random.choice(l)
  with open(page_dir + f"/{une_page}", 'r') as fp:
    lines = len(fp.readlines())
    print(f"la page {une_page} contient {lines} lignes")

la page page1749 contient 1000 lignes
la page page1056 contient 1000 lignes
la page page863 contient 1000 lignes


# Lecture séquentielle d'une table découpée en pages

In [65]:
def lecture_sequentielle_par_page(table):
  page_dir = page_dir_name(table)
  nb_pages = len(os.listdir(page_dir))
  for p in range(1, nb_pages+1) :
    with open(page_dir + f"/page{p}", "r") as f:
      for i, line in enumerate(f):
        tuple_courant = line.strip().split(',')
        yield p, i, tuple_courant

def filtrer_table_par_pages(table, valeur_recherchee):
  for page, position, tuple_courant in lecture_sequentielle_par_page(table):
    attribut0 = int(tuple_courant[0])
    if attribut0 == valeur_recherchee :
      print(f"page {page}, ligne {position} :", tuple_courant)

search = np.random.randint(nb_lignes)
print("valeur recherchée :", search)

t1 = time.time()
filtrer_table_par_pages(TABLE, search)
print("durée :", round(time.time() - t1, 2), "s")

valeur recherchée : 773357
durée : 2.78 s


# Lecture d'un tuple dans une page

Cette fonction retourne le tuple situé dans la page *num_page* et à la position *position*

In [66]:
def lecture_tuple(table, num_page, position):
  page_dir = page_dir_name(table)
  with open(page_dir + f"/page{num_page}", "r") as f:
    lines = f.readlines()
    return lines[position].strip()

t1 = time.time()
print(lecture_tuple(TABLE, 123, 456))
print("done in", round((time.time() - t1)*1000, 1), "ms")

17233030, 9546020, 4178470, 117430, 31540, 530090, 250650,----------------------------------------------------------------------------------------------------
done in 2.4 ms


# Exercice 1 : Créer un index

## Créer un index unique pour l'attribut $a_0$

On sait que $a_0$ est unique.
Une entrée de l'index associe une *clé* à une *valeur* :
*   La *clé* est la valeur du premier attribut.
*   La *valeur* est un **rowid** formé des informations (page, position)



In [67]:
def creation_index_unique(table):
    index = {}
    page_dir = page_dir_name(table)
    nb_pages = len(os.listdir(page_dir))
    for i in range(1, nb_pages + 1):
        with open(page_dir + "/page"+ str(i), 'r') as fp:
            for pos, tuplePage in enumerate(fp):
                index[float(tuplePage.split(",")[0])] = (i, pos)    

    return sortedcontainers.SortedDict(index)  # TODO

t1 = time.time()
INDEX_UNIQUE_a0 = creation_index_unique(TABLE)
print("durée ", round(time.time() - t1, 3), "s")

durée  5.481 s


In [68]:
#vérifier l'index
s = 10 * np.random.randint(nb_lignes)
print(s, INDEX_UNIQUE_a0[s])

14267440 (1727, 971)


## Créer un index non unique pour l'attribut $a_i$

On donne un nom de table et le numéro $i$ de l'attribut $a_i$ de la table.

In [69]:
def creation_index(table, numero_attribut):
    index = {}
    page_dir = page_dir_name(table)
    nb_pages = len(os.listdir(page_dir))
    for i in range(1, nb_pages + 1):
        with open(page_dir + "/page"+ str(i), 'r') as fp:
            for pos, tuplePage in enumerate(fp):
                key = float(tuplePage.split(",")[numero_attribut])
                if key in index.keys():
                    index[key].append((i, pos))
                else:
                    index[key] = [(i, pos)]

    return sortedcontainers.SortedDict(index)


t1 = time.time()
INDEX_a2 = creation_index(TABLE, 2)
print("duree de création de l'index pour l'attribut a2:", round(time.time() - t1, 3), "s")

duree de création de l'index pour l'attribut a2: 6.638 s


In [70]:
# # vérifier l'index
s = 10 * np.random.randint(nb_valeurs_distinctes/4)
print("valeur recherchée :", s)
for r in INDEX_a2[s]:
  print(r)

valeur recherchée : 2768390
(608, 545)
(690, 627)
(703, 713)
(1274, 763)
(1467, 722)
(1721, 604)
(1752, 842)


# Exerccie 2 : Accès par index

## Accès ciblé

On veut retrouver les tuples telq qu'un attribut indexé a une valeur donnée.

### Index unique scan.
On recherche un unique tuple dont l'attribut indexé a une valeur donnée (car l'attribut est unique)

In [71]:
def acces_par_index_unique(index_unique, table, valeur_recherchee):
    page, pos = index_unique[valeur_recherchee] 
    return lecture_tuple(table, page, pos)

s = 10 * np.random.randint(nb_lignes)
print("valeur recherchée :", s)

t1 = time.time()
tuple = acces_par_index_unique(INDEX_UNIQUE_a0, TABLE, s)
print("resulat:", tuple)
print("done in", round((time.time() - t1)*1000, 2), "ms")

valeur recherchée : 19879230
resulat: 19879230, 6042930, 4235030, 608740, 739590, 139530, 80,----------------------------------------------------------------------------------------------------
done in 1.77 ms


### Index scan
Accès pour rechercher les tuples dont l'attribut indexé a une valeur donnée. On suppose que l'attribut n'est pas unique et que plusieurs tuples sont retrouvés

In [72]:
def acces_par_index(index, table, valeur_recherchee):
    listPagePos = index[valeur_recherchee] 
    res = []
    for page, pos in listPagePos:
        res.append(lecture_tuple(table, page, pos))
    return res



s = 10* np.random.randint(nb_lignes/4)
print("valeur recherchée :", s)

t1 = time.time()
for t in acces_par_index(INDEX_a2, TABLE, s):
  print(t)
print("done in", round((time.time() - t1)*1000, 2), "ms")

valeur recherchée : 3998410
3107010, 6645800, 3998410, 327250, 262850, 229190, 205100,----------------------------------------------------------------------------------------------------
12464550, 871810, 3998410, 1255330, 431690, 54840, 9920,----------------------------------------------------------------------------------------------------
2497370, 9945590, 3998410, 495680, 1097460, 73950, 308770,----------------------------------------------------------------------------------------------------
1727290, 6161920, 3998410, 906870, 110990, 61260, 234610,----------------------------------------------------------------------------------------------------
3875870, 6653360, 3998410, 1818940, 411710, 504760, 134920,----------------------------------------------------------------------------------------------------
done in 12.86 ms


## Accès par intervalle
Index range scan



### Accès par intervalle sur un attribut unique
Accès pour rechercher les tuples dont l'attribut indexé est unique et a une valeur comprise dans un intervalle donné.
Indications, votre solution doit prendre en compte les exigences suivantes :
*  Les valeurs recherchées ne sont pas connues à l'avance. On sait seulement qu'elles sont incluses dans un intervalle. Ne pas supposer qu'on recherche des entiers consécutifs.
*  Les bornes de l'intervalle ne sont pas parmi les valeurs existantes de l'attribut. Par exemple, on peut rechercher les valeurs de $a_0$ comprises dans l'intervalle  [23 , 45].



In [73]:
# Exemple pour retrouver la première entrée de l'intervalle [23,45]

indice = INDEX_UNIQUE_a0.bisect_left(23)
print("l'indice de la première clé à retrouver est :", indice)
cle = INDEX_UNIQUE_a0.keys()[indice]
print("la première clé retrouvée est:", cle)
print("la valeur à retrouver est :",  INDEX_UNIQUE_a0[cle])

l'indice de la première clé à retrouver est : 3
la première clé retrouvée est: 30.0
la valeur à retrouver est : (1783, 941)


In [74]:
def acces_intervalle_par_index_unique(index_unique, table, borne_inf, borne_sup):
    posInf = index_unique.bisect_left(borne_inf)
    posSup = index_unique.bisect_left(borne_sup)
    listPagePos = []
    for index in range(posInf, posSup):
        cle = index_unique.keys()[index]
        listPagePos.append(index_unique[cle])
    return listPagePos

s = 10 * np.random.randint(nb_valeurs_distinctes/4)
print("valeur recherchée :", s)

t1 = time.time()
res = acces_intervalle_par_index_unique(INDEX_UNIQUE_a0, TABLE, 23, 45)
print("done in", round(time.time() - t1, 2), "s")
print(res)

valeur recherchée : 73730
done in 0.0 s
[(1783, 941), (828, 168)]


### Accès par intervalle sur un attribut NON unique
Accès pour rechercher les tuples dont l'attribut indexé n'est **pas** unique et a une valeur comprise dans un intervalle donné.
Votre solution doit prendre en compte les mêmes exigences que dans la question précédente.

In [75]:
def acces_intervalle_par_index(index, table, borne_inf, borne_sup):
    posInf = index.bisect_left(borne_inf)
    posSup = index.bisect_left(borne_sup)
    listPagePos = []
    for index_ in range(posInf, posSup):
        cle = index.keys()[index_]
        listPagePos+=index[cle]
    return listPagePos


s = 10 * np.random.randint(nb_valeurs_distinctes / 4)
print("valeur recherchée :", s)

t1 = time.time()
res = acces_intervalle_par_index(INDEX_a2, TABLE, s + 3, s + 33)
print("done in", round(time.time() - t1, 2), "s")
print(res)

valeur recherchée : 1593840
done in 0.0 s
[(647, 461), (695, 621), (1127, 583), (1224, 492), (1365, 257), (1762, 744), (17, 711), (431, 51), (698, 16), (977, 922), (1495, 411), (155, 42), (920, 445), (1978, 65)]


# Exercice 3 : Mise à jour de données




## Modifier la valeur d'un attribut d'un ou plusieurs tuples

Cela correspond à l'insctruction UPDATE table SET ... WHERE ...



### Modification d'un seul tuple

On donne une valeur *v* de l'attribut clé $a_0$. Ajouter 1 à l'attribut $a_1$. Cela correspond à l'instruction

update T
set a1 = a1+10
where a0 = *v*

Après la modification, accéder aux données pour vérifier que le tuple a bien été modifié. Par exemple, invoquer la fonction
acces_par_index_unique(index, table, v)



In [76]:
def update_unique(index, table, v, addNum):
    """ La fonction retourne 1 si nous réussirons à modifier la base de données
                et retourne 0 sinon"""
    if v not in index.keys():
        print("Err : valeur recherchee not existe")
        return 0
    # init 
    page, pos = index[v]
    page_dir = page_dir_name(table)
    file_path = f"{page_dir}/page{page}"
    # lire sur le ficher
    with open(file_path, 'r') as fp:
        lines = fp.readlines()
    # modifier le tuple corresponde
    line = lines[pos].split(",")
    line[1] = str(int(line[1]) + addNum)
    lines[pos] = ",".join(line)
    # ecrire sur le ficher
    with open(file_path, 'w') as fp:
        fp.writelines(lines)
    return 1

# before changing 
valRecherchee = 30
PAGE, POS = INDEX_UNIQUE_a0[valRecherchee]
print("Tuple before changing : ")
print(lecture_tuple(TABLE, PAGE, POS))
# after changing 
update_unique(INDEX_UNIQUE_a0, TABLE, 30,10)
print("Tuple after changing : ")
print(lecture_tuple(TABLE, PAGE, POS))


Tuple before changing : 
30, 3412960, 992630, 2228230, 741740, 516750, 256590,----------------------------------------------------------------------------------------------------
Tuple after changing : 
30,3412970, 992630, 2228230, 741740, 516750, 256590,----------------------------------------------------------------------------------------------------


### Modification de plusieurs tuples

On donne une valeur *v* de l'attribut $a_2$ qui n'est pas unique. Ajouter 1 à l'attribut $a_3$ de tous les tuples pour lesquels $a_2 = v$

update T set a3 = a3+10 where a2=v



In [77]:
def update_plusieurs(index, table, v, addNum):
    """ La fonction retourne le nombre de donnees qu'on modifier dans cette operation 
                et retourne 0 si on ne trouve pas la valeur correspondant"""
    if v not in index.keys():
        print("Err : valeur recherchee not existe")
        return 0
    # init
    listPagePos = index[v]
    page_dir = page_dir_name(table)
    for page, pos in listPagePos:
        file_path = f"{page_dir}/page{page}"
        # lire sur le ficher
        with open(file_path, 'r') as fp:
            lines = fp.readlines()
        # modifier le tuple corresponde
        line = lines[pos].split(",")
        line[3] = str(float(line[3]) + addNum)
        lines[pos] = ",".join(line)
        # ecrire sur le ficher
        with open(file_path, 'w') as fp:
            fp.writelines(lines)
    return len(listPagePos)

# before changing 
valRecherchee = 30
print("The set of Tuple before changing : ")
for PAGE, POS in INDEX_a2[valRecherchee]:
    print(lecture_tuple(TABLE, PAGE, POS))
# after changing 
update_plusieurs(INDEX_a2, TABLE, 30, 10)
print("The set of Tuple after changing : ")
for PAGE, POS in INDEX_a2[valRecherchee]:
    print(lecture_tuple(TABLE, PAGE, POS))


The set of Tuple before changing : 
6205950, 8585550, 30, 1966620, 388790, 617990, 81050,----------------------------------------------------------------------------------------------------
9028020, 4232240, 30, 1507340, 414170, 480850, 54010,----------------------------------------------------------------------------------------------------
10451500, 5134390, 30, 2206910, 689690, 289210, 115540,----------------------------------------------------------------------------------------------------
12116110, 3966910, 30, 1310440, 241660, 371740, 147360,----------------------------------------------------------------------------------------------------
The set of Tuple after changing : 
6205950, 8585550, 30,1966630.0, 388790, 617990, 81050,----------------------------------------------------------------------------------------------------
9028020, 4232240, 30,1507350.0, 414170, 480850, 54010,----------------------------------------------------------------------------------------------------

## Modifier l'index en conséquence lorsque l'attribut modifié est indexé
Comerncer par créer un index sur l'attribut $a_3$

L'attribut $a_3$ étant maintenant indexé, la mise à jour de la question précédente implique d'actualiser l'index sur $a_3$ pour que les rowid des tuples qui contenaient l'ancienne valeur de $a_3$ soient associés à la nouvelle valeur de $a_3$.

In [78]:
def update_plusieurs_index(index, table, va, addNum):
    if v not in index.keys():
        if v not in index.keys():
            print("Err : valeur recherchee not existe")
            return 0
    listOfChange = []
    page_dir = page_dir_name(table)
    
    for page, pos in index[v]:
        file_path = f"{page_dir}/page{page}"
        # lire sur le ficher
        with open(file_path, 'r') as fp:
            lines = fp.readlines()
        # modifier le tuple corresponde
        line = lines[pos].split(",")
        line[3] = str(float(line[3]) + addNum)
        lines[pos] = ",".join(line)
        # changement de a_3
        INDEX_a3[float(line[3])+addNum] = INDEX_a3.pop(float(line[3]))
        listOfChange.append(float(line[3]))
        # ecrire sur le ficher
        with open(file_path, 'w') as fp:
            fp.writelines(lines)
    return listOfChange


INDEX_a3 = creation_index(TABLE, 3)
v = 10 * np.random.randint(nb_valeurs_distinctes / 4)
print("tuple modifis :", v)

t1 = time.time()
updated_tuples = update_plusieurs_index(INDEX_a2, TABLE, v, 10)

#print(updated_tuples)
print("done in", round(time.time() - t1, 3), "s")


tuple modifis : 1526230
done in 0.007 s


# Exercice 4 : Persistence

Dans cette partie, on veut rendre les index persistents en stockant les entrées triées dans des pages. Cela permet d'utiliser les index plus efficacement en réduisant la durée pour les reconstruire.

## Stockage d'un index unique

Proposez une solution pour stocker les entrées **triées** d'un index dans plusieurs pages avec une taille de page fixée (10 000 rowids par page).
Etudier le cas d'un index unique et celui d'un index non unique

In [79]:
def stockIndexUnique(indexUnique):
  # create file path
  indexPath = "indexUniquePages"
  if(os.path.exists(indexPath)):
    sh.rmtree(indexPath)
  os.makedirs(indexPath)

  nbMax = 10000
  lines = []
  # rerwrite stored lines 
  for key in sorted(indexUnique.keys()):
    lines.append(str(key)+', '+str(indexUnique[key][0])+', '+str(indexUnique[key][1]))
  
  maxRange = len(lines)//nbMax
  if len(lines) % nbMax != 0:
    maxRange += 1
  # divide into pages 
  for pageNum in range(maxRange):  
    # créer une page
    with open(indexPath + f"/page{pageNum+1}", "w") as fp:
      borne = pageNum*nbMax
      fp.write('\n'.join(lines[borne:borne+nbMax]))

stockIndexUnique(INDEX_UNIQUE_a0)
     

In [80]:
def stockIndex(index, num):
  # create file path
  indexPath = "indexPages" + str(num)
  if(os.path.exists(indexPath)):
    sh.rmtree(indexPath)
  os.makedirs(indexPath)

  nbMax = 10000
  lines = []
  # rerwrite stored lines 
  for key in sorted(index.keys()):
    for page, pos in index[key]:
      lines.append(str(key)+', '+ str(page) +', ' + str(pos))
  
  maxRange = len(lines)//nbMax
  if len(lines) % nbMax != 0:
    maxRange += 1
  # divide into pages 
  for pageNum in range(maxRange):  
    # créer une page
    with open(indexPath + f"/page{pageNum+1}", "w") as fp:
      borne = pageNum*nbMax
      fp.write('\n'.join(lines[borne:borne+nbMax]))

stockIndex(INDEX_a2, 2)


Montrez que vous pouvez reconstruire les index à partir des entrées stockées dans des pages.

In [81]:
def construireIndexUnique(path):
    indexUnique = {}
    pageMax = len(os.listdir(path))
    for i in range(pageMax):
        with open(path + f"/page{i+1}", "r") as fp:
            lines = fp.readlines()
        for line in lines:
            line = line.split(",")
            indexUnique[float(line[0])] = (int(line[1]), int(line[2]))
    return indexUnique

INDEX_UNIQUE_a0_test = construireIndexUnique("indexUniquePages")
print(INDEX_UNIQUE_a0 == INDEX_UNIQUE_a0_test)


True


In [82]:
def construireIndex(path):
    index = {}
    pageMax = len(os.listdir(path))
    for i in range(pageMax):
        
        with open(path + f"/page{i+1}", "r") as fp:
            lines = fp.readlines()
        for line in lines:
            line = line.split(",")
            if float(line[0]) in index.keys():
                index[float(line[0])].append((int(line[1]), int(line[2])))
            else:
                index[float(line[0])] = [(int(line[1]), int(line[2]))]
    return index

INDEX_a2_test = construireIndex("indexPages2")
print(INDEX_a2 == INDEX_a2_test)


True


## Adapter en conséquence les opérations de modification de l'index

Illustrer le cas :

update T set a3 = a3+0.5 where a1=v

où la nouvelle valeur $a_3' = a_3 + 0.5$ n'est pas déjà présente dans l'index. Il faut donc insérer une nouvelle clé  dans l'index de l'attribut $a_3$.
On suppose qu'il reste de la place dans une page de l'index pour insérer la nouvelle entrée (on peut avoir jusqu'à 12 000 rowids par page d'index).

In [83]:
def updateTest(index_a3, index_a1, val, table):
    if val not in index_a1.keys():
        print("Err : valeur recherchee not existe")
        return 0
    # init 
    listPagePos = index_a1[val]
    page_dir = page_dir_name(table)
    dataToChange = []
    posToChange = []
    
    for page, pos in listPagePos:
        file_path = f"{page_dir}/page{page}"
        # lire sur le ficher
        with open(file_path, 'r') as fp:
            lines = fp.readlines()
        line = lines[pos].split(",")
        dataToChange.append(float(line[3]))
        posToChange.append((page, pos))
    print(dataToChange)
    
    for i, data in enumerate(dataToChange):
        if data+0.5 not in index_a3.keys():
            rowid = posToChange[i]
            index_a3[data].remove(rowid)
            index_a3[data+0.5] = [rowid]
            
    stockIndex(index_a3, 3)
    update_plusieurs(index_a1, table, val, 0.5)

INDEX_a1 = creation_index(TABLE, 1)
updateTest(INDEX_a3, INDEX_a1, 30, TABLE)

[1869430.0, 2388320.0]


# Exercice 5 : Index bitmap
*   Proposer un index ayant une structure matricielle ("bitmap") pour l'attribut $a_5$. Idem pour l'attribut $a_6$.
*   En utilisant les 2 index bitmap, rechercher les tuples de T tels que $a_5 = v_1$ et $a_6 = v_2$ pour deux valeurs $v_1, v_2$ appartenant au domaine de $a_5 \cap a_6$ .




## Facultatif: Index couvrant une requête
* Illustrer les cas vus en TD pour lesquels il est possible d'obtenir le résultat d'une requête sans lire les données de la table lais seulement les index.
